In [ ]:
import pandas as pd
from io import StringIO 
from shapely import wkt
from shapely.geometry import MultiPolygon, Polygon, Point
import json 
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium import LinearColormap
import h3
from math import radians, sin, cos, sqrt, atan2
import matplotlib.pyplot as plt
import pyproj
import sys
from geopy.distance import geodesic

In [57]:
# !pip install "folium>=0.12" matplotlib mapclassify

In [58]:
# read csv and json file
multipolygons_df = pd.read_csv(r'multipolygon_.csv')
multipolygons_df['geometry'] = multipolygons_df['polygon_geometry'].apply(lambda x: wkt.loads(x) if pd.notna(x) else None)

In [59]:
turf_gdf = gpd.GeoDataFrame(multipolygons_df, geometry='geometry', crs="EPSG:4326")
turf_gdf = turf_gdf.drop(columns=['polygon_geometry'])
turf_gdf

,turf_id,total_orders_in_polygon,geometry
0,0577d5b7-951b-4ad4-a9be-12fef82eb9ac,30294,"MULTIPOLYGON (((13.35048 52.48096, 13.34702 52..."
1,975e428d-945b-4f28-808b-382a4b9b5510,13340,"MULTIPOLYGON (((13.28632 52.49963, 13.28823 52..."
2,7755a49a-e332-4444-9c37-fd10164f09ec,10450,"MULTIPOLYGON (((11.51623 48.15007, 11.51636 48..."
3,d68d1739-e0b4-48e5-8a28-8db2eaa9d5fd,7848,"MULTIPOLYGON (((6.7991 51.22948, 6.79554 51.22..."
4,73df1235-55c8-4edb-aa7f-279328d3ea94,6790,"POLYGON ((13.38479 52.50433, 13.38478 52.50338..."
...,...,...,...
88,59703d3f-9eac-42ff-927f-e1c1aca7a2d5,141,"MULTIPOLYGON (((8.43804 49.46231, 8.43977 49.4..."
89,bec7fbe7-647e-4db7-b9ca-f8ecf9a444a5,125,"MULTIPOLYGON (((8.72578 49.41933, 8.72406 49.4..."
90,372eea0b-d4dd-4af0-9521-1957fa901a20,103,"MULTIPOLYGON (((6.97321 50.98761, 6.96868 50.9..."
91,ec2c48a1-310a-458b-a94e-a372e5efcf81,57,"POLYGON ((6.98724 50.90012, 6.98446 50.9023, 6..."


In [60]:
berlin_supermarkets_gdf = gpd.read_file('multipolygon.geojson')
berlin_supermarkets_gdf.head()

,id,@id,access,access:covid19,addr:city,addr:city:fa,addr:country,addr:door,addr:floor,addr:housename,...,website,wheelchair,wheelchair:description,wheelchair:description:de,width,wikidata,wikimedia_commons,wikipedia,zero_waste,geometry
0,relation/7774059,relation/7774059,None,None,Berlin,None,DE,None,None,None,...,https://www.rewe.de,yes,None,None,None,None,None,None,None,"POLYGON ((13.40948 52.55082, 13.40947 52.55078..."
1,relation/15762589,relation/15762589,None,None,Berlin,None,DE,None,None,None,...,None,yes,None,None,None,None,None,None,None,"POLYGON ((13.35252 52.53598, 13.35243 52.53597..."
2,relation/16609777,relation/16609777,None,None,None,None,None,None,None,None,...,https://www.lidl.de/f/berlin/bergstrasse-86.html,yes,None,None,None,None,None,None,None,"POLYGON ((13.32989 52.45836, 13.3299 52.4583, ..."
3,relation/19077225,relation/19077225,None,None,None,None,None,None,None,None,...,None,yes,None,None,None,None,None,None,None,"MULTIPOLYGON (((13.46243 52.42623, 13.4625 52...."
4,way/10672229,way/10672229,None,None,Berlin,None,DE,None,None,None,...,https://www.edeka.de,yes,None,None,None,None,None,None,None,"POLYGON ((13.32837 52.57004, 13.32838 52.57007..."


In [61]:
supermarkets_in_turfs = gpd.sjoin(
    berlin_supermarkets_gdf,
    turf_gdf,               
    how="inner",           
    predicate="within")

len(supermarkets_in_turfs)

65

In [62]:
# viz the turf polygons and supermarkets within the turf

# Berlin city static
berlin_center = (52.5200, 13.4050)
initial_zoom = 11

m = folium.Map(
    location=berlin_center,
    zoom_start=initial_zoom,
    zoom_control=False,      
    scrollWheelZoom=False,   
    dragging=False,          
    doubleClickZoom=False,    
    min_zoom=initial_zoom,  
    max_zoom=initial_zoom)

m = turf_gdf.explore(
    m=m, 
    color="red",
    style_kwds={"fillOpacity": 0.5, "weight": 0.6, "color": "darkred"},
    tooltip=['turf_id', 'total_orders_in_polygon'],
    tooltip_aliases=['Turf ID:', 'Total Orders:'],
    name="Turf Polygons")

if not supermarkets_in_turfs.empty:
    supermarkets_in_turfs.explore(
        m=m,
        marker_type="marker",
        marker_kwds={
            "icon": folium.Icon(color='blue', icon='shopping-cart'),
            "popup": "Supermarket: {name}<br>Brand: {brand}<br>Turf ID: {turf_id}",
            "iconSize": [10, 10]
        },
        tooltip=['name', 'brand', 'turf_id'],
        tooltip_aliases=['Name:', 'Brand:', 'Turf ID:'],
        cmap="blue",
        name="Supermarkets in turf_17",
     )
else:
    print("`supermarkets_in_turfs` df is empty.")
m

In [63]:
import h3
print(f"H3 version: {h3.__version__}") 
print("Available functions:", [func for func in dir(h3) if not func.startswith('_')]) # all the correct and latest functions to use.


H3 version: 4.3.0
Available functions: ['H3BaseException', 'H3CellInvalidError', 'H3DirEdgeInvalidError', 'H3DomainError', 'H3DuplicateInputError', 'H3FailedError', 'H3GridNavigationError', 'H3LatLngDomainError', 'H3MemoryAllocError', 'H3MemoryBoundsError', 'H3MemoryError', 'H3NotNeighborsError', 'H3OptionInvalidError', 'H3PentagonError', 'H3ResDomainError', 'H3ResMismatchError', 'H3Shape', 'H3UndirEdgeInvalidError', 'H3ValueError', 'H3VertexInvalidError', 'LatLngMultiPoly', 'LatLngPoly', 'Literal', 'UnknownH3ErrorCode', 'api', 'are_neighbor_cells', 'average_hexagon_area', 'average_hexagon_edge_length', 'cell_area', 'cell_to_boundary', 'cell_to_center_child', 'cell_to_child_pos', 'cell_to_children', 'cell_to_children_size', 'cell_to_latlng', 'cell_to_local_ij', 'cell_to_parent', 'cell_to_vertex', 'cell_to_vertexes', 'cells_to_directed_edge', 'cells_to_geo', 'cells_to_h3shape', 'child_pos_to_cell', 'compact_cells', 'directed_edge_to_boundary', 'directed_edge_to_cells', 'edge_length', 'g

In [ ]:
# --- Add Hexagonal Density Layer for Supermarkets ---
if not supermarkets_in_turfs.empty:

    h3_resolution = 10

    supermarkets_in_turfs['h3_cell'] = supermarkets_in_turfs.apply(
        lambda row: h3.latlng_to_cell(row.geometry.centroid.y, row.geometry.centroid.x, h3_resolution), axis=1)

    unique_cells = supermarkets_in_turfs['h3_cell'].unique()
    # print(f"Number of unique H3 cells: {len(unique_cells)}")
    # print(f"Sample H3 cells: {unique_cells[:10] if len(unique_cells) > 10 else unique_cells}")

    h3_counts = supermarkets_in_turfs.groupby('h3_cell').size().reset_index(name='supermarket_count')

    hexagons_data = []
    for h3_cell, count in zip(h3_counts['h3_cell'], h3_counts['supermarket_count']):

        boundary_lat_lon = h3.cell_to_boundary(h3_cell)
        boundary_coords = [(lat, lon) for lat, lon in boundary_lat_lon]

        if boundary_coords[0] != boundary_coords[-1]:
            boundary_coords.append(boundary_coords[0])

        # Create WKT coordinates string (lon lat format for WKT)
        wkt_coords = ', '.join([f'{lon} {lat}' for lat, lon in boundary_coords])

        hexagons_data.append({
            'geometry': wkt.loads(f"POLYGON (({wkt_coords}))"),
            'h3_cell': h3_cell,
            'supermarket_count': count})

    hexagons_gdf = gpd.GeoDataFrame(hexagons_data, crs="EPSG:4326")

    min_count = hexagons_gdf['supermarket_count'].min()
    max_count = hexagons_gdf['supermarket_count'].max()

    # Print debug info so you can see what's being created
    print(f"Created {len(hexagons_gdf)} hexagons at resolution {h3_resolution}")
    print(f"Hexagon count range: {min_count} to {max_count}")
    print(f"Hexagon bounds: {hexagons_gdf.total_bounds}")

    # Add hexagons to the map with better visibility
    hexagons_gdf.explore(
        m=m,
        column="supermarket_count", 
        cmap="YlOrRd",  
        style_kwds={"weight": 1, "color": "black", "fillOpacity": 0.7}, 
        tooltip=['h3_cell', 'supermarket_count'],
        tooltip_aliases=['H3 Hexagon ID:', 'Supermarket Count:'],
        name="Supermarket Density (Hexagons)")

    bounds = hexagons_gdf.total_bounds
    padding = 0.05
    southwest = [bounds[1] - padding, bounds[0] - padding]
    northeast = [bounds[3] + padding, bounds[2] + padding]  
    m.fit_bounds([southwest, northeast])

else:
    print("`supermarkets_in_turfs` geo_df is empty.")

folium.LayerControl().add_to(m)
m

Created 56 hexagons at resolution 10
Hexagon count range: 1 to 3
Hexagon bounds: [13.2668704  52.43697313 13.5549083  52.58715237]
